In [ ]:
import sys
# sys.path.append('/home/hileyl/scratch/Projects/PyTorch/3D-ResNets-PyTorch')
sys.path.append('/home/c1435690//Projects/DAIS-ITA/Development/PyTorch/3D-ResNets-PyTorch')
import torch
from models import resnet
from datasets import ucf101
import spatial_transforms, temporal_transforms, target_transforms
import lrp
from tqdm import tqdm

In [2]:
norm_value = 255
ds_mean = [110.63666788 / norm_value, 103.16065604 / norm_value,96.29023126 / norm_value]
ds_std = [38.7568578 / norm_value, 37.88248729 / norm_value,40.02898126 / norm_value]
st = spatial_transforms.Compose([
    spatial_transforms.Scale((112,112)),
    spatial_transforms.ToTensor(255),
    spatial_transforms.Normalize(ds_mean, ds_std)
])
tt = temporal_transforms.LoopPadding(16)
ds = ucf101.UCF101(
    root_path="/media/datasets/Video/UCF-101/jpg",
    annotation_path="/media/datasets/Video/UCF-101/ucf101_01.json",
    subset="validation",
    n_samples_for_each_video=0,
    spatial_transform=st,
    temporal_transform=tt
)


dataset loading [0/3783]
dataset loading [1000/3783]


dataset loading [2000/3783]


dataset loading [3000/3783]


In [3]:
shortcut_type="A"
mdl = resnet.resnet34(num_classes=101, sample_size=112, sample_duration=16, shortcut_type=shortcut_type)
pretrain="/home/hileyl/scratch/Projects/PyTorch/3D-ResNets-PyTorch/results/resnet34_ucf101/save_15.pth"
state = torch.load(pretrain)
sd = state["state_dict"]
if torch.cuda.is_available():
    mdl = torch.nn.DataParallel(mdl).cuda()
mdl.load_state_dict(state_dict=sd, strict=True)
first_lin = True
for l_name, layer in mdl.module.named_children():
    for s_name, sublayer in layer.named_children():
        if (isinstance(sublayer, resnet.BasicBlock)):
            downsample = sublayer.downsample
            in_ = sublayer.conv1.in_channels
            out_ = sublayer.conv1.out_channels
            expans = sublayer.expansion
            stride = sublayer.stride
            if in_ != out_ *  expans or stride != 1:
                sublayer.downsample = lrp.Downsample(in_, out_ * expans, stride=stride, dim=3, shortcut_type=shortcut_type)
                if isinstance(downsample, torch.nn.Sequential):
                    sublayer.downsample.mode.load_state_dict(downsample.state_dict(), strict=True)
        elif (isinstance(sublayer, torch.nn.Linear) and first_lin):
            linear = sublayer
            first_lin = False
    if (isinstance(layer, torch.nn.Linear) and first_lin):
        linear = layer
        setattr(mdl.module, l_name,torch.nn.Sequential(
            lrp.Flatten(),
            layer
        ))
        first_lin = False

/home/hileyl/scratch/Projects/PyTorch/3D-ResNets-PyTorch/models/resnet.py:145: UserWarning: nn.init.kaiming_normal is now deprecated in favor of nn.init.kaiming_normal_.
  m.weight = nn.init.kaiming_normal(m.weight, mode='fan_out')


In [6]:
vid_id = 'v_ApplyEyeMakeup_g01_c01'
samples = []
sample_out = []
mdl = mdl.cuda()
for idx, item in enumerate(ds.data):
    if item["video_id"] != vid_id:
        continue
    sample, target = ds[idx]
    if torch.cuda.is_available():
        sample = sample.cuda()
    sample = torch.unsqueeze(sample,0)
    out = mdl(sample)
    samples.append(sample)
    sample_out.append(out)
vid = torch.cat(samples, 1)
vid_out = torch.cat(sample_out,0).mean(0, keepdim=True)
_, vid_label = vid_out.topk(1,1)
vid_label = vid_label.item()
# samp = torch.unsqueeze(samp, 0).cuda()
# print(samp.shape)
# out = mdl(samp)
# _, pred = out.topk(1,1)
# target["video_id"]

In [5]:
explnr = lrp.DeepTaylor(True)
explanation = torch.zeros_like(vid)
# print(explanation.shape)
print(f"Explaining evidence for {ds.class_names[vid_label]} in {vid_id}")
for sample, out in tqdm(zip(samples, sample_out)):
    filter_out = torch.zeros_like(out)
    filter_out[:, vid_label] = 1.0
    sampleR = explnr.relprop(sample, mdl, out*filter_out)




0it [00:00, ?it/s]

Explaining evidence for Diving in v_ApplyEyeMakeup_g01_c01
DataParallel(
  (module): ResNet(
    (conv1): Conv3d(3, 64, kernel_size=(7, 7, 7), stride=(1, 2, 2), padding=(3, 3, 3), bias=False)
    (bn1): BatchNorm3d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace)
    (maxpool): MaxPool3d(kernel_size=(3, 3, 3), stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): BasicBlock(
        (conv1): Conv3d(64, 64, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1), bias=False)
        (bn1): BatchNorm3d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace)
        (conv2): Conv3d(64, 64, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1), bias=False)
        (bn2): BatchNorm3d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): BasicBlock(
        (conv1): Conv3d(64, 64, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 

torch.Size([1, 512, 1, 4, 4])
torch.Size([1, 512, 1, 4, 4])
torch.Size([1, 512, 1, 4, 4])
torch.Size([1, 512, 1, 4, 4])
torch.Size([1, 512, 1, 4, 4])


RuntimeError: Given groups=1, weight of size [512, 256, 1, 1, 1], expected input[1, 512, 1, 4, 4] to have 256 channels, but got 512 channels instead